In [3]:
import os

In [4]:
os.chdir("..")

In [5]:
%pwd

'd:\\Nikku\\End to End Project\\Kidney-Disease-Classification'

In [6]:
# Prepare Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

In [7]:
# Configurartion Manager

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        training = self.config.model_trainer
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        
        create_directories([
            Path(training_data)
        ])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        return model_trainer_config
         

In [9]:
# Update the Components

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [17]:
class Training:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
    
        )
        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_generator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path:Path, model = tf.keras.Model):
        model.save(path)
        
        
    def train(self):
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.step_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
            
        )

In [18]:
# Pipeline
try:
    config = ConfigurationManager()
    training_config = config.get_model_trainer_config()
    training = Training(config=training_config)
    training.get_base_model()  
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e


[2023-10-13 15:41:19,801: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-13 15:41:19,803: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-13 15:41:19,804: INFO: common: Created directory at: artifacts]
[2023-10-13 15:41:19,804: INFO: common: Created directory at: artifacts\data_ingestion\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
Epoch 1/10
622/622 [==============================] - 448s 720ms/step - loss: 13.6499 - accuracy: 0.4222 - val_loss: 10.4567 - val_accuracy: 0.3327
Epoch 2/10
622/622 [==============================] - 462s 743ms/step - loss: 7.1050 - accuracy: 0.5440 - val_loss: 5.5662 - val_accuracy: 0.5101
Epoch 3/10
622/622 [==============================] - 483s 776ms/step - loss: 5.1694 - accuracy: 0.6091 - val_loss: 5.6208 - val_accuracy: 0.5129
Epoch 4/10
622/622 [==============================] - 464s 747ms/step - loss: 4.6009 - accuracy: 0